# 1. szorgalmi feladat

## D

**Sudoku feladványok megoldása**

Hozzuk létre a **SudokuSolver** osztályt, mely Sudoku feladványok megoldására alkalmaz különböző szabályokat!

Az osztálynak egy publikus metódusa van, **solve()** néven. A függvény egy (9,9) alakú NumPy integer tömböt kap paraméterként, melyben a tábla aktuális állapota van eltárolva, az üres helyek nullákkal vannak feltöltve. A **solve()** függvény feladata a különböző, megvalósított szabályok alkalmazása a táblán, a szabályok segítségével kikövetkeztetett új számjegyek beírása a táblába és a tábla visszaadása. Az egyes szabályok konkrét megvalósításai természetesen nyugodtan kerülhetnek más, privát metódusokba, melyeket a **solve()** függvény hív meg sorban.

A tesztkód példányosítja az osztályt, majd 100 darab feladványon sorban teszteli a megoldást. A tesztkód a **solve()** függvényt hívja kezdetben az aktuális feladvány tábla eredeti, megoldatlan állapotával, majd egészen addig ismételten hívja a függvényt a részben kitöltött táblát átadva neki, amíg a tábla nincs teljesen kitöltve, vagy amíg a **solve()** függvény képes új számjegyet beírni a táblába. Ha egy **solve()** hívás nem változtatja meg a táblát (esetleg hibásan változtatja azt) és a tábla nem megoldott, a tesztelő továbblép a következő feladványra és rögzíti, hogy az adott feladványt nem sikerült megoldani. Amennyiben egy **solve()** hívás a megoldott táblát adja vissza, a tesztelő továbblép a következő feladványra és rögzíti, hogy az adott feladványt sikerült megoldani. A megoldott feladványok számát a tesztelő a futtatás végeztével kiírja.

A szabályokat a következő oldalon találjátok: http://hodoku.sourceforge.net/en/tech_singles.php. 

- A *Hidden Single* szabály olyan sor/oszlop/négytzetblokk - számjegy párosokat keres, ahol az adott sor/oszlop/négytzetblokkban a számjegy pontosan egy helyre írható be. Ha létezik ilyen, a számjegyet beírjuk a megfelelő helyre.

- A *Naked Single* módszer olyan cellákat keres, ahová pontosan egy számjegy írható be. Ha létezik ilyen, a számjegyet beírjuk a megfelelő helyre.

Ez a két szabály, helyesen megvalósítva, a 100 feladványból 79-et képes megoldani. A többi feladvány megoldásához további szabályok megvalósítása szükséges.

**Megjegyzés:** A weboldalon található *Full House/Last Digit* szabályt nem érdemes implementálni, hiszen ez a fenti két szabály egy speciális esete, tehát nem fog további feladványokat megoldani.

**Kikötés:** A szorgalmi házi feladatot, az 1. kötelező házi feladatsor B) és C) feladatához hasonlóan, NumPy segítségével, Python ciklusok és annak megfelelő szerkezetek nélkül, hatékony módon kell megoldani. Az ezeknél a feladatoknál felsorolt kikötések itt is érvényesek.


### Megoldás

In [1]:
import numpy as np 

# Your solution -> 
def concatenate_per_row(a, b):
    """
    Creates a matrix of all possible row and column combination of
    two matrices.
    """
    m1, n1 = a.shape
    m2, n2 = b.shape

    out = np.zeros((m1, m2, n1 + n2), dtype=a.dtype)
    out[:, :, :n1] = a[:, None, :]
    out[:, :, n1:] = b

    return out.reshape(m1 * m2, -1)


def bincount2d(a):
    """
    Numpy bincount on a 2D matrix.
    """
    bins = np.max(a) + 1

    count = np.zeros(shape=[len(a), bins], dtype=np.int64)
    indexing = (np.ones_like(a).T * np.arange(len(a))).T

    np.add.at(count, (indexing, a), 1)

    return count


class SudokuSolver:
    SIZE = 9
    BLOCK_SIZE = 3
    assert SIZE % BLOCK_SIZE == 0

    def __get_available_values(self, board):
        """
        :param board: the sudoku board
        :return: An indicator matrix of possible values, with a dimension of (board_rows, board_cols, value_cnt), where
                 the last dimension indicates that a value with a given index can be placed to that field
        """

        row_col_combos = concatenate_per_row(board, board.T)

        blocks = board.T.reshape(SudokuSolver.SIZE, -1, SudokuSolver.BLOCK_SIZE) \
            .swapaxes(0, 1) \
            .reshape(SudokuSolver.SIZE, SudokuSolver.SIZE)

        blocks_per_element = blocks.repeat(SudokuSolver.BLOCK_SIZE, axis=0) \
            .reshape(SudokuSolver.BLOCK_SIZE, SudokuSolver.SIZE, -1) \
            .repeat(SudokuSolver.BLOCK_SIZE, axis=0) \
            .reshape(-1, SudokuSolver.SIZE)

        present_per_element = np.concatenate((row_col_combos, blocks_per_element), axis=1)
        present_indicator_mx = bincount2d(present_per_element)[:, 1:].astype(bool)

        is_empty_element = (board.reshape(-1, 1) == 0)

        possible_values = ~present_indicator_mx & is_empty_element

        return possible_values.reshape(SudokuSolver.SIZE, SudokuSolver.SIZE, -1)

    def __apply_transform_on_all_dims(row_fun):
        """
        Applies a transformation that is implemented to work on a row
        to all dimensions of the board: rows, columns and blocks.
        """

        def inner(self, board, available_val_mx, *args, **kwargs):
            # Apply on rows
            board, available_val_mx = row_fun(self, board, available_val_mx, *args, **kwargs)

            # Apply on columns
            # since we can already apply on rows, we just need to transpose
            # the matrix, apply it and transpose the result
            board, available_val_mx = row_fun(self, board.T, available_val_mx.swapaxes(0, 1), *args, **kwargs)
            board, available_val_mx = board.T, available_val_mx.swapaxes(0, 1)

            # Apply on houses
            # we are transforming each house in a row, apply the rule on the rows
            # and transform back the result onto the houses
            def board_transform(b): return b.reshape(SudokuSolver.BLOCK_SIZE, SudokuSolver.BLOCK_SIZE,
                                                     -1, SudokuSolver.BLOCK_SIZE) \
                                            .swapaxes(1, 2) \
                                            .reshape(SudokuSolver.SIZE, SudokuSolver.SIZE)

            def available_val_transform(a): return a.reshape(SudokuSolver.BLOCK_SIZE, SudokuSolver.BLOCK_SIZE,
                                                             SudokuSolver.BLOCK_SIZE, -1, SudokuSolver.SIZE) \
                                                    .swapaxes(1, 2) \
                                                    .reshape(SudokuSolver.SIZE, SudokuSolver.SIZE, -1)

            board_block, available_block = board_transform(board), available_val_transform(available_val_mx)

            board_block, available_block = row_fun(self, board_block, available_block, *args, **kwargs)
            board, available_val_mx = board_transform(board_block), available_val_transform(available_block)

            return board, available_val_mx

        return inner

    def apply_naked_single(self, board, available_val_mx):
        """
        Applies the naked single rule (http://hodoku.sourceforge.net/en/tech_singles.php#n1).
        """

        naked_single_loc_mx = self.__get_available_values(board) \
                                  .sum(axis=2) == 1  # the given field has only 1 possible value

        # Since only one is possible, only 1 value will be true along axis=2,
        # argmax will return it's position, which is equal to (possible_value - 1)
        naked_single_values = np.argmax(available_val_mx, axis=2) + 1

        # if at a given position we have a naked single get it's value from naked_single_values
        # else keep the value already in the board
        new_board = np.where(naked_single_loc_mx, naked_single_values, board)

        # available_val_mx isn't changed
        return new_board, available_val_mx

    @__apply_transform_on_all_dims
    def apply_hidden_single(self, board, available_val_mx):
        """
        Applies the hidden single rule (http://hodoku.sourceforge.net/en/tech_singles.php#h1).
        """

        # swap 'column' <--> 'value' axis, so values will be align to
        # the column, they should be placed into
        av_val_mx_view = available_val_mx.swapaxes(1, 2)

        # 1. Sum the rows of the indicator mx., if only one value is possible,
        # then it is a hidden single in that row.
        # 2. Apply logical AND so from mx keep only the rows that are hidden singles.
        hidden_single_loc_mx = (av_val_mx_view.sum(axis=2) == 1).reshape(SudokuSolver.SIZE, -1, 1) & av_val_mx_view

        # 1. Replace bool values with the number they represent, according to
        # it's index, if it is a hidden single. otherwise with 0.
        # 2. Sum along axis=1 so we get the rows of the transformed board, in
        # places where there is a hidden single.
        hidden_single_values = np.where(hidden_single_loc_mx,
                                        np.arange(1, SudokuSolver.SIZE + 1).reshape(-1, 1),
                                        0).sum(axis=1)

        # where there isn't a hidden single, we want to keep the original
        new_board = np.where(hidden_single_values != 0, hidden_single_values, board)

        # available_val_mx isn't changed
        return new_board, available_val_mx

    @__apply_transform_on_all_dims
    def apply_naked_pairs(self, board, available_val_mx, k=2):
        """
        Applies the naked pairs rule (http://hodoku.sourceforge.net/en/tech_naked.php).

        :param k: Which rule to apply. k=2 - 'naked pair'; k=3 - 'naked triple'; k=4 - 'naked quadruple'
        """

        # swap 'column' <--> 'value' axis, so values will be align to
        # the column, they should be placed into
        av_val_mx_view = available_val_mx.swapaxes(1, 2)

        # Create a copy of the indicator matrix.
        # It is used to store the location of the naked pairs.
        copy = av_val_mx_view.copy()

        # rows where there is only two possible value
        only_k_possible_rows = copy & (copy.sum(axis=1) == k)[:, None, :]

        # if in a row not exactly k values are possible set every possible value
        # for that row to True (to be able to apply logical AND correctly later)
        only_k_possible_rows.swapaxes(1, 2)[~np.any(only_k_possible_rows, axis=1)] = True

        # will be only True in columns where there are k possible values
        copy = np.where(np.all(only_k_possible_rows, axis=2)[:, :, None],
                        copy & (copy.sum(axis=1) == k)[:, None, :],
                        False)

        # Filter according to the rule.
        # Do not apply on rows where there isn't exactly k possible number.
        only_k_poss_values = copy & (copy.sum(axis=1) == k)[:, None, :]

        # The value pairs must appear at least in k columns
        at_least_in_k_cols = only_k_poss_values & (only_k_poss_values.sum(axis=2) >= k)[:, :, None]

        # Rows where there are valid naked pairs
        v = np.any(at_least_in_k_cols, axis=2)[:, :, None]

        # if a valid pair, remove all occurrence of the pair elsewhere in the row
        # else keep the original.
        new_available_val_mx = np.where(v, at_least_in_k_cols, av_val_mx_view).swapaxes(1, 2)

        # board isn't changed
        return board, new_available_val_mx

    def solve(self, board):
        available_val_mx = self.__get_available_values(board)

        # apply 'naked pairs', 'naked triple', 'naked quadruple'
        board, available_val_mx = self.apply_naked_pairs(board, available_val_mx, 2)
        board, available_val_mx = self.apply_naked_pairs(board, available_val_mx, 3)
        board, available_val_mx = self.apply_naked_pairs(board, available_val_mx, 4)

        # apply naked single
        board, available_val_mx = self.apply_naked_single(board, available_val_mx)

        # apply hidden single
        board, available_val_mx = self.apply_hidden_single(board, available_val_mx)

        return board



### Tesztek

In [2]:
import urllib.request
import pickle
sudoku_url = "https://nipg12.inf.elte.hu/~vavsaai@nipg.lab/annbsc22_p1_hw1/sudoku.pkl"
sudokus = pickle.load(urllib.request.urlopen(sudoku_url))

solved_count=0

for t in sudokus:
    solver = SudokuSolver()
    puzzle, solution = t["puzzle"], t["solution"]
    current_board = puzzle.copy()
    
    is_solved, found_candidates = False, True
    while (not is_solved) & found_candidates:
        result = solver.solve(current_board)
        if np.all(result == solution):
            solved_count+=1
            is_solved = True
        elif np.all(result == current_board):
            found_candidates = False
        
        assert np.all(result[current_board != 0] == current_board[current_board != 0]), "Do not change already inserted elements!" 

        current_board = result

print("{:d} of {:d} sudokus were solved".format(solved_count, len(sudokus)))


80 of 100 sudokus were solved
